In [ ]:
# | default_exp _code_generator.app_skeleton_generator

In [ ]:
# | export

from typing import *
import time
import json
import ast
from pathlib import Path
from collections import defaultdict

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.chat import CustomAIChat, ValidateAndFixResponse
from faststream_gen._code_generator.helper import (
    write_file_contents,
    validate_python_code,
    retry_on_error,
)
from faststream_gen._code_generator.prompts import APP_SKELETON_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    STEP_LOG_DIR_NAMES,
    APPLICATION_FILE_PATH,
    LOGS_DIR_NAME
)

In [ ]:


from tempfile import TemporaryDirectory

import pytest

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.constants import OpenAIModel
from faststream_gen._code_generator.helper import mock_openai_create

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

CODE_CONTAINS_IMPLEMENTATION_ERROR = "Error: The response contains code implementation. Rewrite the skeleton code without implementing the business logic for the functions. Ensure the new code has only google styled docstring describing the business logic step by step and raise NotImplementedError()"

In [ ]:
# | export


def _has_function_implementation(
    node: Union[ast.AsyncFunctionDef, ast.FunctionDef]
) -> bool:
    return len(node.body) == 2 and isinstance(node.body[-1], ast.Raise)


def _check_response_for_implementation(response: str) -> List[str]:
    parsed = ast.parse(response)
    function_nodes = [
        node
        for node in ast.walk(parsed)
        if isinstance(node, ast.AsyncFunctionDef) or isinstance(node, ast.FunctionDef)
    ]
    ret_val = (
        []
        if all(_has_function_implementation(node) for node in function_nodes)
        else [CODE_CONTAINS_IMPLEMENTATION_ERROR]
    )
    return ret_val

In [ ]:
fixture = '''
import asyncio
import json

import aiohttp

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., examples=[50000.0], description="Current price of cryptocurrency in USD")
    crypto_currency: str = Field(..., examples=["BTC"], description="The cryptocurrency")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


async def fetch_crypto_price(url: str) -> dict:
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()


async def get_and_publish_crypto_price(
    crypto_currency: str, logger: Logger, context: ContextRepo, time_interval: int = 2
) -> None:
    """
    While app_is_running variable inside context is True, repeat the following process:
        Retrieve the current cryptocurrency price by sending a GET request to the appropriate URL.
        Extract the price and crypto_currency from the response JSON.
        Create a CryptoPrice object with the retrieved data.
        Publish the CryptoPrice object to the 'new_crypto_price' topic, using the utf-8 encoded crypto_currency as the partition key.
        Asynchronously sleep for the specified time_interval.
    """
    raise NotImplementedError()


@app.on_startup
async def app_setup(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to True - we will use this variable as running loop condition
    """
    raise NotImplementedError()


@app.on_shutdown
async def shutdown(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to False

    Get all executed tasks from context and wait for them to finish
    """
    raise NotImplementedError()


@app.after_startup
async def publish_crypto_price(logger: Logger, context: ContextRepo):
    """
    Create asynchronous tasks for executing get_and_publish_crypto_price function.
    Run this process for Bitcoin and Ethereum.
    Put all executed tasks into a list and set it as a global variable in the context (It is needed so we can wait for these tasks at app shutdown)
    """
    raise NotImplementedError()
'''

actual = _check_response_for_implementation(fixture)
print(actual)
assert actual == [CODE_CONTAINS_IMPLEMENTATION_ERROR]

['Error: The response contains code implementation. Rewrite the skeleton code without implementing the business logic for the functions. Ensure the new code has only google styled docstring describing the business logic step by step and raise NotImplementedError()']


In [ ]:
fixture = '''
import asyncio
import json

import aiohttp

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(
        ..., examples=[50000.0], description="Current price of cryptocurrency in USD"
    )
    crypto_currency: str = Field(
        ..., examples=["BTC"], description="The cryptocurrency symbol"
    )


async def fetch_crypto_price(
    url: str, crypto_currency: str, logger: Logger, context: ContextRepo
) -> None:
    """
    Fetches the current cryptocurrency price from the provided URL and publishes it to the 'new_crypto_price' topic.

    Instructions:
    1. Send a GET request to the provided URL.
    2. Retrieve the current price and cryptocurrency symbol from the response JSON.
    3. Create a CryptoPrice object with the retrieved data.
    4. Publish the CryptoPrice object to the 'new_crypto_price' topic, using the crypto_currency as the partition key.
    5. Sleep for 2 seconds.
    """
    raise NotImplementedError()


@app.on_startup
async def app_setup(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to True - we will use this variable as running loop condition
    """
    raise NotImplementedError()


@app.on_shutdown
async def shutdown(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to False

    Get all executed tasks from context and wait them to finish
    """
    raise NotImplementedError()


@app.after_startup
async def fetch_crypto_prices(logger: Logger, context: ContextRepo):
    """
    Create asynchronous tasks for executing fetch_crypto_price function.
    Run this process for Bitcoin (BTC) and Ethereum (ETH).
    Put all executed tasks to list and set it as global variable in context (It is needed so we can wait for these tasks at app shutdown)
    """
    raise NotImplementedError()
'''

actual = _check_response_for_implementation(fixture)
print(actual)
assert actual == []

[]


In [ ]:
# | export


def _validate_response(
    response: str, output_directory: str, **kwargs: Dict[str, Any]
) -> List[str]:
    target_file_name = Path(output_directory) / APPLICATION_FILE_PATH
    write_file_contents(str(target_file_name), response)
    code_import_errors = validate_python_code(str(target_file_name))
    
    if len(code_import_errors) != 0:
        return code_import_errors
    
    return _check_response_for_implementation(response)

In [ ]:
fixture = """
import os
import invalid_module
def say_hello():
    print("hello")
"""

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = ["ModuleNotFoundError: No module named 'invalid_module'"]

    print(actual)
    assert actual == expected

["ModuleNotFoundError: No module named 'invalid_module'"]


In [ ]:

fixture = """
import os
def say_hello():
    print("hello")
"""

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = [CODE_CONTAINS_IMPLEMENTATION_ERROR]

    print(actual)
    assert actual == expected

['Error: The response contains code implementation. Rewrite the skeleton code without implementing the business logic for the functions. Ensure the new code has only google styled docstring describing the business logic step by step and raise NotImplementedError()']


In [ ]:
fixture = '''
import os
def say_hello():
    """This is a docstring"""
    raise NotImplementedError()
'''

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = []

    print(actual)
    assert actual == expected

[]


In [ ]:
fixture = '''
import asyncio
import json

import aiohttp

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(
        ..., examples=[50000.0], description="Current price of cryptocurrency in USD"
    )
    crypto_currency: str = Field(
        ..., examples=["BTC"], description="The cryptocurrency symbol"
    )


async def fetch_crypto_price(
    url: str, crypto_currency: str, logger: Logger, context: ContextRepo
) -> None:
    """
    Fetches the current cryptocurrency price from the provided URL and publishes it to the 'new_crypto_price' topic.

    Instructions:
    1. Send a GET request to the provided URL.
    2. Retrieve the current price and cryptocurrency symbol from the response JSON.
    3. Create a CryptoPrice object with the retrieved data.
    4. Publish the CryptoPrice object to the 'new_crypto_price' topic, using the crypto_currency as the partition key.
    5. Sleep for 2 seconds.
    """
    raise NotImplementedError()


@app.on_startup
async def app_setup(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to True - we will use this variable as running loop condition
    """
    raise NotImplementedError()


@app.on_shutdown
async def shutdown(context: ContextRepo):
    """
    Set all necessary global variables inside ContextRepo object:
        Set app_is_running to False

    Get all executed tasks from context and wait them to finish
    """
    raise NotImplementedError()


@app.after_startup
async def fetch_crypto_prices(logger: Logger, context: ContextRepo):
    """
    Create asynchronous tasks for executing fetch_crypto_price function.
    Run this process for Bitcoin (BTC) and Ethereum (ETH).
    Put all executed tasks to list and set it as global variable in context (It is needed so we can wait for these tasks at app shutdown)
    """
    raise NotImplementedError()
'''

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = []

    print(actual)
    assert actual == expected

[]


In [ ]:
fixture = '''
import asyncio
import json

import aiohttp

from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import ContextRepo, FastStream, Logger
from faststream.kafka import KafkaBroker

broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


class CryptoPrice(BaseModel):
    price: NonNegativeFloat = Field(..., examples=[50000.0], description="Current price of cryptocurrency in USD")
    crypto_currency: str = Field(..., examples=["BTC"], description="The cryptocurrency")


async def fetch_crypto_price(url: str) -> dict:
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()


async def get_crypto_price(crypto_currency: str) -> CryptoPrice:
    url = f"https://api.coinbase.com/v2/prices/{crypto_currency}-USD/spot"
    response = await fetch_crypto_price(url)
    price = response["data"]["amount"]
    return CryptoPrice(price=price, crypto_currency=crypto_currency)


@app.on_startup
async def app_setup(context: ContextRepo):
    context.crypto_currencies = ["BTC", "ETH"]


@app.on_shutdown
async def shutdown(context: ContextRepo):
    context.app_is_running = False
    await asyncio.gather(*context.tasks)


async def fetch_and_publish_crypto_price(
    crypto_currency: str, logger: Logger, context: ContextRepo, interval: int = 2
) -> None:
    while context.app_is_running:
        crypto_price = await get_crypto_price(crypto_currency)
        await broker.publish(
            topic="new_crypto_price",
            key=crypto_currency.encode("utf-8"),
            value=crypto_price.json().encode("utf-8"),
        )
        await asyncio.sleep(interval)


@app.after_startup
async def publish_crypto_price(logger: Logger, context: ContextRepo):
    context.tasks = []
    for crypto_currency in context.crypto_currencies:
        task = asyncio.create_task(
            fetch_and_publish_crypto_price(crypto_currency, logger, context)
        )
        context.tasks.append(task)


if __name__ == "__main__":
    asyncio.run(app.run())
'''

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = [CODE_CONTAINS_IMPLEMENTATION_ERROR]

    print(actual)
    assert actual == expected

['Error: The response contains code implementation. Rewrite the skeleton code without implementing the business logic for the functions. Ensure the new code has only google styled docstring describing the business logic step by step and raise NotImplementedError()']


In [ ]:
# | export


@retry_on_error()  # type: ignore
def _generate(
    model: str,
    prompt: str,
    app_description_content: str,
    total_usage: List[Dict[str, int]],
    output_directory: str,
    **kwargs,
) -> Tuple[str, List[Dict[str, int]]]:
    app_generator = CustomAIChat(
        params={
            "temperature": 0.2,
        },
        model=model,
        user_prompt=prompt,
        #         semantic_search_query=app_description_content,
    )
    app_validator = ValidateAndFixResponse(app_generator, _validate_response)
    validator_result = app_validator.fix(
        app_description_content,
        total_usage,
        STEP_LOG_DIR_NAMES["skeleton"],
        str(output_directory),
        **kwargs,
    )

    return (
        (validator_result, True)
        if isinstance(validator_result[-1], defaultdict)
        else validator_result
    )

In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some valid prompt"
app_description_content = "some valid app description"
total_usage = [defaultdict(int)]


test_response = """
print("some valid skeleton code")
"""

with TemporaryDirectory() as d:
    with mock_openai_create(test_response):
        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, app_description_content, total_usage, d
        )
        assert is_valid_skeleton_code == True
        print("OK")

[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the message class is a derived class of BaseModel from pydantic.

        Example of a Valid message class:
            class Pet(BaseModel):
            

In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some invalid prompt"
app_description_content = "some invalid app description"
total_usage = [defaultdict(int)]

test_response = """
print("some valid skeleton code"
"""

with TemporaryDirectory() as d:
    with mock_openai_create(test_response):
        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, app_description_content, total_usage, d
        )
        print(is_valid_skeleton_code)
        assert is_valid_skeleton_code == False
        print("OK")

[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the message class is a derived class of BaseModel from pydantic.

        Example of a Valid message class:
            class Pet(BaseModel):
            

[INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: '(' was never closed (application.py, line 2)
[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the messa

[INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: '(' was never closed (application.py, line 2)
[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the messa

In [ ]:
# | export


def generate_app_skeleton(
    validated_description: str,
    output_directory: str,
    model: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> Tuple[List[Dict[str, int]], bool]:
    """Generate skeleton code for the new FastStream app from the application description

    Args:
        code_gen_directory: The directory containing the generated files.
        total_usage: list of token usage.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The total token used to generate the FastStream code
    """
    logger.info("==== Description to Skeleton Generation ====")
    with yaspin(
        text=f"Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...",
        color="cyan",
        spinner="clock",
    ) as sp:
        prompt = APP_SKELETON_GENERATION_PROMPT.replace(
            "==== RELEVANT EXAMPLES GOES HERE ====", f"\n{relevant_prompt_examples}"
        )

        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, validated_description, total_usage, output_directory
        )
        sp.text = ""
        if is_valid_skeleton_code:
            message = f" ✔ FastStream app skeleton code generated."
        else:
            message = " ✘ Error: Failed to generate a valid application skeleton code."
            sp.color = "red"

        sp.ok(message)
        return total_usage, is_valid_skeleton_code

In [ ]:
fixture_description = "Some valid description"

relevant_prompt_examples = "Some valid examples"

fixture_response = 'print("hi")'

with mock_openai_create(fixture_response):
    with TemporaryDirectory() as d:
        output_file = Path(d) / APPLICATION_FILE_PATH

        usage, is_valid_skeleton_code = generate_app_skeleton(
            fixture_description, d, OpenAIModel.gpt3.value, [], relevant_prompt_examples
        )

        assert Path(output_file).exists()

        with open(output_file, "r", encoding="utf-8") as f:
            actual = f.read()

        print(actual)
        assert actual == fixture_response
        
        assert is_valid_skeleton_code

[INFO] __main__: ==== Description to Skeleton Generation ====
⠋ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the mess

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


In [ ]:
fixture_description = "Some invalid description"

relevant_prompt_examples = "Some invalid examples"

fixture_response = 'i am a not a python code'

with mock_openai_create(fixture_response):
    with TemporaryDirectory() as d:
        output_file = Path(d) / APPLICATION_FILE_PATH
        usage, is_valid_skeleton_code = generate_app_skeleton(
            fixture_description, d, OpenAIModel.gpt3.value, [], relevant_prompt_examples
        )
        print(is_valid_skeleton_code)
        assert not is_valid_skeleton_code

[INFO] __main__: ==== Description to Skeleton Generation ====
⠋ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

3. Whenever you are creating a message class while generating Faststream skeleton and the application code, make sure the mess

[INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: invalid syntax (application.py, line 1)
[INFO] faststream_gen._code_generator.helper: Attempt 0 failed. Restarting step.
⠹ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... [INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" sta

[INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: invalid syntax (application.py, line 1)
[INFO] faststream_gen._code_generator.helper: Attempt 1 failed. Restarting step.
⠼ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... [INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" sta

⠴ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... [INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: invalid syntax (application.py, line 1)
[INFO] faststream_gen._code_generator.helper: Attempt 2 failed. Restarting step.
 ✘ Error: Failed to generate a valid application skeleton code.                      
False


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
